## 1. Connect to Azure OpenAI LLM
Initialize and connect to the Azure OpenAI service using credentials from environment variables to enable chat completions.

In [ ]:
import os
import dotenv


In [ ]:
dotenv.load_dotenv()

In [ ]:
#connect to llm
from openai import AzureOpenAI
from dotenv import load_dotenv


client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("OPENAI_BASE_URL"),
    api_version="2024-08-01-preview"
)

response = client.chat.completions.create(   
  model="gpt-4o",#  Replace with your actual deployment name from Azure Portal
  messages=[
    {"role": "user", "content": "This is a test."}
  ]
)

print(response.model_dump_json(indent=2))

## 2. Install Azure Search Documents Library
Install the required `azure-search-documents` package to interact with Azure AI Search service.
- pip install azure-search-documents

## 3. Connect to Azure AI Search Service
Initialize the SearchClient with endpoint, index name, and API key from environment variables to enable queries against the Azure Search index.

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

service_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = os.environ["AZURE_SEARCH_INDEX_NAME"]
key = os.environ["AZURE_SEARCH_API_KEY"]

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

## 4. Search and Display Results
Execute a search query against the Azure Search index and display all results with their fields and values.

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

# Semantic search with reranking
results = search_client.search(
    search_text="ROBERT AUDI",
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="default",  # or your configured semantic config name
    top=5  # number of results to return
)

print("Semantic Search Results...")
result_list = list(results)
print(f"Number of results: {len(result_list)}")

if result_list:
    print("\nFirst result:")
    print(result_list[0])
    
    for i, result in enumerate(result_list):
        print(f"\n--- Result {i+1} ---")
        for key, value in result.items():
            print(f"{key}: {value}")
else:
    print("No results found")

## 8. Compare Normal Search vs Semantic Search

### What You'll See

This cell performs **both search types** on the same query and compares the results:

- **🔵 Normal Keyword Search**: Matches exact words in your documents
- **🟢 Semantic Search**: Understands meaning and context, ranks by relevance

### Which One is Better?

It depends on your use case:
- **Normal Search**: Fast, good for exact matches ("John Smith")
- **Semantic Search**: Smarter, better at understanding intent ("Who is the main character?")

**For RAG applications**: Semantic Search usually gives better results because it understands meaning, not just keywords.

### Try It!

Enter a search query below and compare the results from both search methods.

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

# Define which fields you want to display
# those fields and data just for test that's why the ain't that good 
IMPORTANT_FIELDS = [
    'metadata_storage_name',      # File name
    'content',                     # Main content
    'keyphrases',                  # Key phrases
    'people',                       # People mentioned
]

user_query = input("What do you want to search for? = ")

print(f"\n{'='*70}")
print(f"QUERY: {user_query}")
print(f"{'='*70}\n")

# ============================================================================
# 1. NORMAL (KEYWORD) SEARCH
# ============================================================================
print("🔵 NORMAL KEYWORD SEARCH")
print("-" * 70)

normal_results = search_client.search(search_text=user_query, top=5)
normal_list = list(normal_results)

print(f"Results found: {len(normal_list)}\n")

if normal_list:
    for i, result in enumerate(normal_list, 1):
        print(f"  Result {i}:")
        for key, value in result.items():
            # Only show important fields
            if key in IMPORTANT_FIELDS:
                # Truncate long values for readability
                if isinstance(value, str):
                    value_str = value[:150] + "..." if len(value) > 150 else value
                else:
                    value_str = str(value)[:150] + "..." if len(str(value)) > 150 else str(value)
                print(f"    {key}: {value_str}")
        print()
else:
    print("  No results found.\n")

# ============================================================================
# 2. SEMANTIC SEARCH
# ============================================================================
print("\n" + "="*70)
print("🟢 SEMANTIC SEARCH (AI-Powered Ranking)")
print("-" * 70)

semantic_results = search_client.search(
    search_text=user_query,
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="default",
    top=5
)

semantic_list = list(semantic_results)

print(f"Results found: {len(semantic_list)}\n")

if semantic_list:
    for i, result in enumerate(semantic_list, 1):
        print(f"  Result {i}:")
        for key, value in result.items():
            # Only show important fields
            if key in IMPORTANT_FIELDS:
                # Truncate long values for readability
                if isinstance(value, str):
                    value_str = value[:150] + "..." if len(value) > 150 else value
                else:
                    value_str = str(value)[:150] + "..." if len(str(value)) > 150 else str(value)
                print(f"    {key}: {value_str}")
        print()
else:
    print("  No results found.\n")

# ============================================================================
# 3. COMPARISON
# ============================================================================
print("\n" + "="*70)
print("📊 COMPARISON")
print("-" * 70)
print(f"Normal Search Results:    {len(normal_list)} documents")
print(f"Semantic Search Results:  {len(semantic_list)} documents")
print("\n💡 Note:")
print("  - Normal Search: Matches keywords in documents")
print("  - Semantic Search: Understands meaning and context better")
print("="*70 + "\n")

# single turn-chat


## Query Rewriting for Better Search Results

Before searching, we optimize the user's natural language question into a search-optimized query. This helps Azure AI Search find more relevant philosophy documents by:
- Extracting key philosophical terms and concepts
- Removing conversational filler words
- Expanding with related philosophical terminology

In [ ]:
# System message for query rewriting - optimized for philosophy searches
QUERY_REWRITE_SYSTEM_MESSAGE = """You are a query optimization assistant for a philosophy knowledge base (Cambridge Dictionary of Philosophy).

Your task: Transform the user's natural language question into an optimized search query for Azure AI Search.

Rules:
1. Extract key philosophical terms, concepts, and philosopher names
2. Remove conversational words (what, how, can you, please, etc.)
3. Include relevant philosophical synonyms or related terms
4. Keep it concise (3-10 words maximum)
5. Return ONLY the optimized query - no explanations, no quotes, no extra text

Examples:
- User: "What did Kant think about morality?" → "Kant ethics moral philosophy categorical imperative"
- User: "Can you explain what existentialism means?" → "existentialism Sartre Heidegger existence meaning"
- User: "Who is Plato and what are his main ideas?" → "Plato forms idealism Republic Socrates"
- User: "What's the difference between ethics and morality?" → "ethics morality moral philosophy normative"
- User: "Tell me about free will" → "free will determinism libertarianism compatibilism"

Remember: Output ONLY the optimized search query, nothing else."""


def rewrite_query(user_message: str) -> str:
    """
    Takes a user's natural language question and returns an optimized search query.
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": QUERY_REWRITE_SYSTEM_MESSAGE},
            {"role": "user", "content": user_message}
        ],
        max_tokens=50,  # Keep responses short
        temperature=0.3  # Lower temperature for more consistent outputs
    )
    return response.choices[0].message.content.strip()



# test the query llm 

In [ ]:

# Test the query rewriter
test_questions = [
    "What is the meaning of life according to philosophers?",
    "Can you explain Nietzsche's ideas?",
    "What's epistemology about?"
]

print("🔄 Query Rewriting Examples:\n")
for q in test_questions:
    optimized = rewrite_query(q)
    print(f"Original:  {q}")
    print(f"Optimized: {optimized}")
    print("-" * 50)

# Full RAG pipeline with query rewriting


In [ ]:

user_message = input("Ask a philosophy question: ")

# Step 1: Rewrite the query for better search
print(f"\n📝 Original question: {user_message}")
optimized_query = rewrite_query(user_message)
print(f"🔄 Optimized query: {optimized_query}")

# Step 2: Search with the optimized query
semantic_results = search_client.search(
    search_text=optimized_query,  # Use optimized query instead of raw user input
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="default",
    select=[ "keyphrases"],  # Exclude 'content'
    top=2
)
semantic_list = list(semantic_results)
print(f" 📚 Found {len(semantic_list)} relevant documents")


In [ ]:
def to_markdown(results):## turn the search to markdown 
    if not results:
        return "No relevant documents found."
    blocks = []
    for i, doc in enumerate(results, 1):
        blocks.append(
            f"### Document {i}\n"
            f"- **filename:** {doc.get('metadata_storage_name', 'N/A')}\n"
            f"- **people:** {', '.join(doc.get('people', [])) if doc.get('people') else 'N/A'}\n"
            f"- **keyphrases:** {', '.join(doc.get('keyphrases', [])) if doc.get('keyphrases') else 'N/A'}\n\n"
            f"**content:**\n{doc.get('content', 'N/A')}\n"
        )
    return "\n---\n".join(blocks)



In [ ]:
# Adjust max_content_length based on your needs:
# - 2000 chars = ~500 tokens (very concise)
# - 4000 chars = ~1000 tokens (good balance)
# - 8000 chars = ~2000 tokens (more context)

context_md = to_markdown(semantic_list)
print(f"Context length: {len(context_md)} characters (~{len(context_md)//4} tokens)")

In [ ]:
# System message for the RAG response generation
RAG_SYSTEM_MESSAGE = """You are a knowledgeable philosophy assistant powered by the Cambridge Dictionary of Philosophy.

Your role: Synthesize search results from the knowledge base to provide accurate, helpful answers to user questions.

Guidelines:
1. **Use ONLY the provided context** - Do not make up information or use external knowledge
2. **Be accurate** - Quote or paraphrase directly from the source documents when possible
3. **Be concise** - Provide clear, focused answers without unnecessary filler
4. **Cite sources** - Reference document names or philosophers mentioned in the context
5. **Acknowledge limitations** - If the context doesn't contain enough information, say so honestly
6. **Explain philosophical concepts** - Make complex ideas accessible without oversimplifying
7. **Stay neutral** - Present philosophical views objectively without personal bias

Response format:
- Start with a direct answer to the question
- Provide supporting details from the context
- Mention relevant philosophers or works if available
- Keep responses well-structured and readable

If the context is insufficient or irrelevant to the question, respond with:
"I couldn't find enough information in the knowledge base to answer this question accurately."
"""

In [ ]:
# Send to LLM with proper system message and structured prompt
response = client.chat.completions.create(   
    model="gpt-4o",
    messages=[
        {"role": "system", "content": RAG_SYSTEM_MESSAGE},
        {"role": "user", "content": f"**Context from knowledge base:**\n{context_md}\n\n**User Question:** {user_message}"}
    ],
    temperature=0.7,  # Balanced creativity and accuracy
    max_tokens=500    # Reasonable response length
)

print("🤖 Answer:\n")
print(response.choices[0].message.content)

# Multi-Turn RAG Chat

Now let's implement a **multi-turn conversation** with memory. This allows the assistant to:
- Remember previous questions and answers
- Handle follow-up questions like "Tell me more" or "What about his other ideas?"
- Maintain context across the conversation

## How it works:
1. We keep a `conversation_history` list that stores all messages
2. Each new question triggers a search → retrieval → response cycle
3. The full conversation history is sent to the LLM for context

In [ ]:
# Multi-turn RAG system message (enhanced for conversation context)
MULTI_TURN_SYSTEM_MESSAGE = """You are a knowledgeable philosophy assistant powered by the Cambridge Dictionary of Philosophy.

Your role: Engage in multi-turn conversations about philosophy, using search results to provide accurate answers.

Guidelines:
1. **Use the provided context** - Base your answers on the search results provided
2. **Remember conversation history** - Reference previous exchanges when relevant
3. **Handle follow-ups naturally** - Understand questions like "tell me more", "what else?", "and his views on X?"
4. **Be accurate and cite sources** - Reference philosophers and works from the context
5. **Acknowledge limitations** - If context is insufficient, say so honestly
6. **Stay coherent** - Maintain consistency with your previous answers

If asked a follow-up without new context, use your previous answers and conversation history.
If the context is insufficient, respond: "I couldn't find enough information to answer this accurately."
"""

# Initialize conversation history
conversation_history = []

def multi_turn_rag(user_message: str, conversation_history: list) -> str:
    """
    Process a user message in a multi-turn RAG conversation.
    
    Args:
        user_message: The user's current question
        conversation_history: List of previous messages [{"role": "user/assistant", "content": "..."}]
    
    Returns:
        The assistant's response
    """
    # Step 1: Rewrite query for better search
    optimized_query = rewrite_query(user_message)
    print(f"🔄 Optimized query: {optimized_query}")
    
    # Step 2: Search with optimized query
    semantic_results = search_client.search(
        search_text=optimized_query,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        top=3
    )
    semantic_list = list(semantic_results)
    print(f"📚 Found {len(semantic_list)} relevant documents")
    
    # Step 3: Convert results to markdown context
    context_md = to_markdown(semantic_list)
    
    # Step 4: Build messages with history
    messages = [{"role": "system", "content": MULTI_TURN_SYSTEM_MESSAGE}]
    
    # Add conversation history
    messages.extend(conversation_history)
    
    # Add current user message with context
    current_message = f"**New context from search:**\n{context_md}\n\n**User Question:** {user_message}"
    messages.append({"role": "user", "content": current_message})
    
    # Step 5: Get LLM response
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.7,
        max_tokens=500
    )
    
    assistant_response = response.choices[0].message.content
    
    # Step 6: Update conversation history (store simplified version)
    conversation_history.append({"role": "user", "content": user_message})
    conversation_history.append({"role": "assistant", "content": assistant_response})
    
    return assistant_response

print("✅ Multi-turn RAG function initialized!")

## Interactive Chat Loop

Run the cell below to start a multi-turn conversation. Type your questions and see how the assistant remembers context!

- Type `quit` or `exit` to end the conversation
- Type `clear` to reset conversation history
- Try follow-up questions like "Tell me more" or "What about X?"

In [ ]:
# Interactive multi-turn chat loop
print("🧠 Multi-Turn Philosophy RAG Chat")
print("=" * 50)
print("Commands: 'quit'/'exit' to end, 'clear' to reset memory")
print("=" * 50)

# Reset conversation history for new session
conversation_history = []

while True:
    user_input = input("\n👤 You: ").strip()
    
    # Handle special commands
    if user_input.lower() in ['quit', 'exit']:
        print("\n👋 Goodbye! Conversation ended.")
        break
    
    if user_input.lower() == 'clear':
        conversation_history = []
        print("🗑️ Conversation history cleared!")
        continue
    
    if not user_input:
        print("Please enter a question.")
        continue
    
    # Process the message
    print(f"\n{'─' * 50}")
    response = multi_turn_rag(user_input, conversation_history)
    print(f"\n🤖 Assistant:\n{response}")
    print(f"{'─' * 50}")
    print(f"📝 Memory: {len(conversation_history)//2} exchanges stored")

## Test Memory with Follow-up Questions

Try this conversation flow to test memory:

1. **First question:** "Who is Kant?"
2. **Follow-up:** "What are his main ideas?"
3. **Another follow-up:** "Tell me more about the categorical imperative"

The assistant should understand that "his" refers to Kant from the previous context!

In [ ]:
# View current conversation history
print("📜 Current Conversation History:")
print("=" * 50)

if not conversation_history:
    print("No conversation history yet. Start chatting above!")
else:
    for i, msg in enumerate(conversation_history):
        role = "👤 User" if msg["role"] == "user" else "🤖 Assistant"
        content = msg["content"][:200] + "..." if len(msg["content"]) > 200 else msg["content"]
        print(f"\n{role}:\n{content}")
        
print(f"\n{'=' * 50}")
print(f"Total exchanges: {len(conversation_history)//2}")

In [ ]:
# LLM-as-a-Judge System Message for RAG Evaluation
# Evaluates: Faithfulness, Relevance, Completeness, and Coherence

LLM_JUDGE_SYSTEM_MESSAGE = """You are an expert evaluator for RAG (Retrieval-Augmented Generation) systems.

Your task: Evaluate the quality of an AI assistant's response based on the provided context and user question.

## Evaluation Criteria (Score 1-5 for each):

### 1. FAITHFULNESS (Is the answer grounded in the context?)
- 5: Fully supported by context, no hallucinations
- 4: Mostly supported, minor unsupported claims
- 3: Partially supported, some claims lack evidence
- 2: Mostly unsupported, significant hallucinations
- 1: Completely fabricated or contradicts context

### 2. RELEVANCE (Does it answer the user's question?)
- 5: Directly and completely answers the question
- 4: Answers the question with minor gaps
- 3: Partially answers, misses key aspects
- 2: Tangentially related, mostly off-topic
- 1: Completely irrelevant to the question

### 3. COMPLETENESS (Is the answer thorough?)
- 5: Comprehensive, covers all relevant aspects from context
- 4: Good coverage, minor omissions
- 3: Covers basics, misses important details
- 2: Superficial, significant gaps
- 1: Extremely incomplete or empty

### 4. COHERENCE (Is it well-structured and clear?)
- 5: Excellent structure, clear and easy to follow
- 4: Good structure, minor clarity issues
- 3: Adequate structure, some confusion
- 2: Poor structure, hard to follow
- 1: Incoherent or nonsensical

## Output Format (JSON):
```json
{
    "faithfulness": {"score": X, "reason": "brief explanation"},
    "relevance": {"score": X, "reason": "brief explanation"},
    "completeness": {"score": X, "reason": "brief explanation"},
    "coherence": {"score": X, "reason": "brief explanation"},
    "overall_score": X.X,
    "summary": "One sentence overall assessment"
}
```

IMPORTANT: 
- Be strict but fair in your evaluation
- Always output valid JSON only - no extra text
- Base faithfulness ONLY on the provided context
- The overall_score is the average of all four scores
"""